In [ ]:
# !pip install -U "jax[tpu]" -f https://storage.googleapis.com/jax-releases/libtpu_releases.html
# !pip install vartests
!pip install numpyro
# !pip install flax
# !pip install aeppl==0.0.17
# !pip install blackjax
# !pip install jax==0.4.26

In [ ]:
%matplotlib inline

import numpy as np
import scipy as sp
from pathlib import Path
from tqdm import tqdm
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns


import pymc as pm
import arviz as az

import os
import graphviz

import jax
from jax.lib import xla_bridge
# import blackjax
import pymc.sampling_jax

import pytensor
from pymc.pytensorf import collect_default_updates
import pytensor.tensor as pt

import cv2
import numpy as np
from google.colab import files

print(f'numpy version: {np.__version__}')
print(f'scipy version: {sp.__version__}')
print(f'arviz version: {az.__version__}')
print(f'pymc version: {pm.__version__}')
# print(f'jax version: {jax.__version__}')
# print(f'platform: {xla_bridge.get_backend().platform}')
# print(f'device: {jax.devices()}')

numpy version: 1.25.2
scipy version: 1.11.4
arviz version: 0.15.1
pymc version: 5.10.4


In [ ]:
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/My Drive/course_3

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/course_3


In [ ]:
encoding_name = 'cp1251'
hour_ahead_df1 = pd.read_csv('oes_hour_01012023_01062023.csv', sep=';', encoding=encoding_name)
hour_ahead_df2 = pd.read_csv('oes_hour_02062023_01022024.csv', sep=';', encoding=encoding_name)
hour_ahead_df = pd.concat([hour_ahead_df2, hour_ahead_df1], ignore_index=True)
rsv_hour_df = hour_ahead_df

res_name = 'Индекс равновесных цен на покупку электроэнергии, руб./МВт*ч'
rsv_hour_df.reset_index(drop=True, inplace=True)
rsv_hour_df.drop(columns=['ОЭС', 'Объем полного планового потребления, МВт*ч', 'Объем планового производства, МВт*ч',
                          'Индекс равновесных цен на продажу электроэнергии, руб.МВт*ч', 'Максимальный индекс равновесной цены, руб.МВт*ч',
                          'Минимальный индекс равновесной цены, руб.МВт*ч', 'Unnamed: 9'], inplace=True)
rsv_hour_df.rename(columns={'Дата':'Date','Час':'Hour', res_name:'Price, RUB/MWh.'}, inplace=True)

df = rsv_hour_df.copy()
df.loc[:, 'Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Hour'].astype('str'), format='%d.%m.%Y %H')
df.sort_values(by=['Datetime'], ignore_index=True, inplace=True)
df['Diff'] = np.log(df['Price, RUB/MWh.']).diff()
df.dropna(inplace=True)
df.drop(columns=['Date', 'Hour'], inplace=True)

# Плановые отключения электроэнергии в начале августа 2023 года, в эти дни невозможны адекватные прогназирования.
index = df[df['Diff'] > df['Diff'].quantile(.9999)].index
df.drop(index, inplace=True)

print(hour_ahead_df1.shape, hour_ahead_df2.shape, hour_ahead_df.shape)
df.head()

(3648, 10) (5880, 10) (9528, 3)


,"Price, RUB/MWh.",Datetime,Diff
1,1123.70,2023-01-01 01:00:00,-0.048506
2,1164.18,2023-01-01 02:00:00,0.035390
3,1114.10,2023-01-01 03:00:00,-0.043970
4,1091.61,2023-01-01 04:00:00,-0.020393
5,1092.11,2023-01-01 05:00:00,0.000458


In [ ]:
encoding_name = 'cp1251'
hour_ahead_df1 = pd.read_csv('oes_hour_01012023_01062023.csv', sep=';', encoding=encoding_name)
hour_ahead_df2 = pd.read_csv('oes_hour_02062023_01022024.csv', sep=';', encoding=encoding_name)
rsv_hour_df = pd.concat([hour_ahead_df2, hour_ahead_df1], ignore_index=True)

res_name = 'Индекс равновесных цен на покупку электроэнергии, руб./МВт*ч'

rsv_hour_df.rename(columns={'Дата':'Date','Час':'Hour', res_name:'Price, RUB/MWh.'}, inplace=True)
rsv_hour_df.drop(columns=['Unnamed: 9'], inplace=True)
rsv_hour_df.sort_values(by=['Date', 'Hour'], ignore_index=True, inplace=True)
rsv_hour_df['Diff'] = -np.log(rsv_hour_df['Price, RUB/MWh.']).diff(periods=-1)
rsv_hour_df.fillna(0, inplace=True)
rsv_hour_df['Datetime'] = pd.to_datetime(rsv_hour_df['Date'], format="%d.%m.%Y")
rsv_hour_df.drop(columns=['ОЭС', 'Объем полного планового потребления, МВт*ч', 'Объем планового производства, МВт*ч',
                          'Индекс равновесных цен на продажу электроэнергии, руб.МВт*ч', 'Максимальный индекс равновесной цены, руб.МВт*ч',
                          'Минимальный индекс равновесной цены, руб.МВт*ч', 'Datetime', 'Date', 'Hour'], inplace=True)

hour_temp_df = pd.read_csv('Moscow,Russia 2023-01-01 to 2024-02-01.csv')
# hour_temp_df.drop(index=0, inplace=True)
hour_temp_df.reset_index(inplace=True)
hour_temp_df.rename(columns={'datetime':'Datetime','temp':'Temp'}, inplace=True)
hour_temp_df['log_Temp'] = np.log(np.abs(hour_temp_df['Temp']))
# hour_temp_df['Diff_temp'] = hour_temp_df['Temp'].diff()
hour_temp_df['Datetime'] = hour_temp_df['Datetime'].astype('datetime64[ns]')
hour_temp_df = hour_temp_df[['Datetime', 'Temp', 'log_Temp']]

df = pd.concat([hour_temp_df, rsv_hour_df], axis=1)

# Плановые отключения электроэнергии в начале августа 2023 года, в эти дни невозможны адекватные прогназирования.
index = df[df['Diff'] > df['Diff'].quantile(.9999)].index
df.drop(index, inplace=True)

df.loc[:, 'Weekday'] = df['Datetime'].dt.day_name()
one_hot = pd.get_dummies(df['Weekday'], dtype='int')
df = pd.concat([df, one_hot[['Saturday', 'Sunday', 'Monday']]], axis=1)
df.drop(columns=['Weekday'], inplace=True)
df.reset_index(drop=True, inplace=True)

print(f'Данные цены электроэнергии с 01.01.2023 по 01.06.2023: {hour_ahead_df1.shape}')
print(f'Данные цены электроэнергии с 02.06.2023 по 01.02.2024: {hour_ahead_df2.shape}')
print(f'Данные цены электроэнергии с 01.01.2023 по 01.02.2024: {rsv_hour_df.shape}')
print(f'Исторический климат г. Москвы 01.01.2023 по 01.02.2024: {hour_temp_df.shape}')
df.head()

Данные цены электроэнергии с 01.01.2023 по 01.06.2023: (3648, 10)
Данные цены электроэнергии с 02.06.2023 по 01.02.2024: (5880, 10)
Данные цены электроэнергии с 01.01.2023 по 01.02.2024: (9528, 2)
Исторический климат г. Москвы 01.01.2023 по 01.02.2024: (9528, 3)


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Datetime,Temp,log_Temp,"Price, RUB/MWh.",Diff,Saturday,Sunday,Monday
0,2023-01-01 00:00:00,0.4,-0.916291,1179.55,-0.048506,0,1,0
1,2023-01-01 01:00:00,0.4,-0.916291,1123.70,0.035390,0,1,0
2,2023-01-01 02:00:00,1.0,0.000000,1164.18,-0.043970,0,1,0
3,2023-01-01 03:00:00,1.0,0.000000,1114.10,-0.020393,0,1,0
4,2023-01-01 04:00:00,1.0,0.000000,1091.61,0.000458,0,1,0


In [ ]:
sum(df['log_Temp'].isna())

0

### ARX model:

In [ ]:
!pip install gekko

In [ ]:
from gekko import GEKKO
t = np.arange(len(df['Datetime']))
t = df['Datetime']
u = df['Temp']
y = df['Diff']
m = GEKKO()

# system identification
na = 2 # output coefficients
nb = 2 # input coefficients
y_obs,p,K = m.sysid(t,u,y,na,nb,pred='meas')
y = np.exp(df['Diff'])

print(f'RMSE: {mean_squared_error(y, y_obs, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y, y_obs)}')
print(f'MAE: {mean_squared_error(y, y_obs)}')
print(f'R^2: {r2_score(y, y_obs)}')

RMSE: 1.0145212294212373
MAPE: 1.0006232696012372
MAE: 1.029253324946379
R^2: -62.39198870231231


In [ ]:

with pm.Model() as arx_model:
    # Коэффициенты авторегрессии
    beta = pm.Normal('beta', mu=0, sigma=1.0)

    # Коэффициент экзогенной переменной
    gamma = pm.Normal('gamma', mu=0, sigma=1.0)

    # Шум
    sigma = pm.HalfNormal('sigma', sigma=1.0)

    # Экзогенная переменная
    u = pm.Data('u', df['log_Temp'], mutable=False)

    # Авторегрессионная компонента
    y_ar = pm.AR('y_ar', beta, sigma=sigma, observed=df['Diff'])

    # Полная модель
    y = y_ar + gamma * u

    # Наблюдаемые данные
    y_obs = pm.Normal('y_obs', mu=y, sigma=sigma, observed=df['Diff'])


/usr/local/lib/python3.10/dist-packages/pymc/distributions/timeseries.py:558: UserWarning: Initial distribution not specified, defaulting to `Normal.dist(0, 100, shape=...)`. You can specify an init_dist manually to suppress this warning.
  warnings.warn(


In [ ]:
y = df['Diff']
u = df['log_Temp']

with pm.Model() as arx_model:
    # Priors for AR and X coefficients
    ar_coefs = pm.Normal('ar_coefs', mu=0, sigma=1, shape=2)
    x_coef = pm.Normal('x_coef', mu=0, sigma=1)

    # Initialize latent states
    ar1 = pm.Normal('ar1', mu=0, sigma=1)
    ar2 = pm.Normal('ar2', mu=0, sigma=1)

    # Build ARX model
    y_pred = pm.Deterministic('y_pred', ar_coefs[0] * ar1 + ar_coefs[1] * ar2 + x_coef * u)
    y_obs = pm.Normal('y_obs', mu=y_pred, sigma=1, observed=y)

In [ ]:
rng = np.random.RandomState(1234)
with arx_model:

    arx_trace = pm.sample_prior_predictive(500, random_seed=rng)

arx_prior_predictive = az.extract(arx_trace, group='prior_predictive')

In [ ]:
arx_trace

Inference data with groups:
	> prior
	> prior_predictive
	> observed_data
	> constant_data

In [ ]:
rng = np.random.RandomState(1234)
with arx_model:
    arx_trace.extend(pm.sampling.jax.sample_numpyro_nuts(random_seed=rng, chain_method='parallel'))

ar_posterior = az.extract(arx_trace)
# ar_posterior['exp_h'] = np.exp(ar_posterior['h'])

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
with arx_model:
    arx_trace.extend(pm.sample_posterior_predictive(arx_trace, random_seed=rng))

arx_posterior_predictive = az.extract(arx_trace, group='posterior_predictive')

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

# y_obs = np.exp(arx_trace.posterior_predictive['y_obs'].mean(('chain', 'draw')))
y = np.exp(df['Diff'])

print(f'RMSE: {mean_squared_error(y, y_obs, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y, y_obs)}')
print(f'MAE: {mean_squared_error(y, y_obs)}')
print(f'R^2: {r2_score(y, y_obs)}')

RMSE: 1.0145212294212373
MAPE: 1.0006232696012372
MAE: 1.029253324946379
R^2: -62.39198870231231


### AR model:

In [ ]:
with pm.Model() as ar_model:
    beta = pm.Normal('beta', mu=0, sigma=1.0)
    y_obs = pm.AR('y_obs', beta, sigma=1.0, observed=df['Diff'])

In [ ]:
rng = np.random.RandomState(1234)
with ar_model:
    ar_trace = pm.sample_prior_predictive(500, random_seed=rng)

ar_prior_predictive = az.extract(ar_trace, group='prior_predictive')

In [ ]:
rng = np.random.RandomState(1234)
with ar_model:
    ar_trace.extend(pm.sampling.jax.sample_numpyro_nuts(random_seed=rng, chain_method='parallel'))

ar_posterior = az.extract(ar_trace)
# ar_posterior['exp_h'] = np.exp(ar_posterior['h'])

In [ ]:
ar_trace

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> prior_predictive
	> observed_data

In [ ]:
with ar_model:
    ar_trace.extend(pm.sample_posterior_predictive(ar_trace, random_seed=rng))

ar_posterior_predictive = az.extract(ar_trace, group='posterior_predictive')

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

y_obs = np.exp(ar_trace.posterior_predictive['y'].mean(('chain', 'draw')))
y = np.exp(df['Diff'])

print(f'RMSE: {mean_squared_error(y, y_obs, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y, y_obs)}')
print(f'MAE: {mean_squared_error(y, y_obs)}')
print(f'R^2: {r2_score(y, y_obs)}')

RMSE: 0.12624694520945814
MAPE: 0.07253114934457093
MAE: 0.015938291174719926
R^2: -0.022134601839244095


### Naive model:

In [ ]:
with pm.Model() as naive_model:
    mu = pm.math.cos(df['Diff'])

    sigma = pm.HalfNormal('sigma')
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=df['Diff'])


In [ ]:
rng = np.random.RandomState(1234)
with naive_model:
    naive_trace = pm.sample_prior_predictive(500, random_seed=rng)

naive_prior_predictive = az.extract(naive_trace, group='prior_predictive')

In [ ]:
rng = np.random.RandomState(1234)
with naive_model:
    naive_trace.extend(pm.sampling.jax.sample_numpyro_nuts(random_seed=rng, chain_method='parallel'))

naive_posterior = az.extract(naive_trace)

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
with naive_model:
    naive_trace.extend(pm.sample_posterior_predictive(naive_trace, random_seed=rng))

naive_posterior_predictive = az.extract(naive_trace, group='posterior_predictive')

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

y_obs = np.exp(naive_trace.posterior_predictive['y_obs'].mean(('chain', 'draw')))
y = np.exp(df['Diff'])

print(f'RMSE: {mean_squared_error(y, y_obs, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y, y_obs)}')
print(f'MAE: {mean_squared_error(y, y_obs)}')
print(f'R^2: {r2_score(y, y_obs)}')

RMSE: 1.706622572544533
MAPE: 1.722518571456324
MAE: 2.9125606051185198
R^2: -185.7847024383277
